In [67]:
#  Import des bibliothèques 
from bs4 import BeautifulSoup
import requests
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from random import randint
from time import sleep
import os


L'American Association of Individual Investors (AAII) réalise depuis 1987 une enquête hebdomadaire sur l'opinion des investisseurs concernant le marché financier pour les six prochains mois. Les résultats permettent de donner un sentiment de marché (parfois utile pour réaliser des décisions d'investissements car les cours de certains actifs sont dépendant du sentiment de marché) et les résultats sont répartis en 3 catégories :
- Bull (investisseurs optimistes)
- Bear (investisseurs pessimistes)
- Neutral (investisseurs neutres)

In [68]:
def load_results_from_file():
    #  On vérifie si le fichier existe déjà dans le dossier actuel 
    file_name = 'sentiment_results_survey.csv'
    # Si c'est le cas on lit le fichier 
    if os.path.exists(file_name):
        df = pd.read_csv(file_name,parse_dates=False)
        df = df.sort_index(ascending=False)
        return df
    else:
        return None


In [69]:
def retrieve_survey_results(url):
    try:
        #  On crée une session de navigation web
        session = requests.Session()
        response = session.get(url)
        session_cookies = session.cookies
        cookies_dictionary = session_cookies.get_dict()
        
        #  On récupère les paramètres de cookies
        list_from_dict = []
        for key in cookies_dictionary:
            list_from_dict.append(key)
            list_from_dict.append(cookies_dictionary[key])

        #  Build cookie
        cookie='{}={}; {}={}'.format(list_from_dict[2], list_from_dict[3], list_from_dict[0], list_from_dict[1])
        headers = {
            'Host': 'www.aaii.com',
            'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0',
            'DNT': '1',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'ru,en-US;q=0.9,en;q=0.8,tr;q=0.7',
            'Cookie': '{}'.format(cookie),
        }
        # On récupère le texte contenu dans l'url 
        html_text = requests.get(url, headers=headers,timeout=20).text
        return html_text
    except:
        print('Fetch request failed')
        return None


In [70]:
def process_results(html_text):
    # La libraire BeautifulSoup permet de parser le texte que nous avons extrait de la page web
    soup = BeautifulSoup(html_text, "html.parser")
    # On recherche tous les éléments html <tr> qui indiquent chaque ligne du tableau que nous cherchons à importer 
    results = soup.find_all('tr',{'align': 'center'})
    results_df = pd.DataFrame()
    for i in range(1, len(results) - 1):
        # Pour chaque ligne du tableau on récupère chaque élément html <td> qui indique une cellule du tableau
        elements = results[i].find_all('td')
        # On place dans une liste la date, puis l'élément de la colonne Bullish, puis l'élément de la colonne Neutral et enfin celui de la colonne Bearish
        reported_date =elements[0].getText().strip().replace(':','')

        bullish=elements[1].getText().strip().replace('%','')
        neutral=elements[2].getText().strip().replace('%','')
        bearish=elements[3].getText().strip().replace('%','')
        # On crée un dataframe à partir de la liste puis on le concatène avec le dataframe results_df
        result_row = pd.DataFrame({"reported_date":[reported_date],"bullish":[bullish],"bearish":[bearish],"neutral":[neutral]})
        results_df = pd.concat([results_df, result_row], axis=0, ignore_index = True)
    results_df = results_df.sort_index(ascending=False)
    results_df.iloc[::-1].reset_index(drop=True)
    
        
    return results_df 



In [71]:
def plot_results(df):
    fig = make_subplots(rows=1, cols=1)

    #  On graph les données extraites de la page web
    fig.add_trace(go.Line(x = df['reported_date'], y = df['neutral'], line=dict(color="blue", width=1), name="Neutral"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df['reported_date'], y = df['bullish'], line=dict(color="green", width=1), name="Bullish"), row = 1, col = 1) 
    fig.add_trace(go.Line(x = df['reported_date'], y = df['bearish'], line=dict(color="red", width=1), name="Bearish"), row = 1, col = 1) 
    
    fig.update_layout(
        title={'text':'AAII Sentiment Results', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()


In [72]:


try:
        
    results_df = load_results_from_file()
    #  On inverse l'odre des lignes
    results_df = results_df[::-1]

except :
    
    if results_df is None:
        print('Fichier contenant les résultats non trouvé - import des résultats à partir du web')
        #  import à partir du web
        url = 'https://www.aaii.com/sentimentsurvey/sent_results'
        html_text = retrieve_survey_results(url)
        print(html_text)
        #  Vérification de la protection anti-robots
        if "Incapsula incident ID" in html_text:
            print('Import bloqué par la protection robot - réessayer ultérieurement')
        else:
            #  Mise au format des résultats
            results_df = process_results(html_text)
            print(results_df)
            #  Enregistrement des résultats dans un csv
            results_df.to_csv('sentiment_results_survey.csv')
    #results_df = results_df.loc[::-1].reset_index(drop=True).head()

#  Graphique 
if results_df is not None:
    plot_results(results_df)

